<a href="https://colab.research.google.com/github/johnlocke333/xray_image_classification/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Preparation**


In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip -uq "/content/drive/My Drive/xray.zip" -d "/content/drive/My Drive/xray"

In [3]:
# Directory path
train_data_dir = '/content/drive/MyDrive/xray/chest_xray/chest_xray/train'
test_data_dir = '/content/drive/MyDrive/xray/chest_xray/chest_xray/test'
valid_data_dir = '/content/drive/MyDrive/xray/chest_xray/chest_xray/val'


In [4]:
def get_reshape_images(dir, bs):
# Get all the data in the directory data/validation (5216 images), and reshape them
  generator = ImageDataGenerator().flow_from_directory(
          dir, target_size=(64, 64), batch_size=bs)

# Create the datasets
  images, labels = next(generator)
  return images, labels

In [5]:
train_images, train_labels = get_reshape_images(train_data_dir, 5216)

Found 5216 images belonging to 2 classes.


In [6]:
test_images, test_labels = get_reshape_images(test_data_dir, 624)

Found 624 images belonging to 2 classes.


In [7]:
valid_images, valid_labels = get_reshape_images(valid_data_dir, 16)

Found 16 images belonging to 2 classes.


In [51]:
 def reshape_input_image_label(images, m, labels):
  img_unrow = images.reshape(m, -1)
  labels_final = labels.T[[1]]
  return img_unrow, labels_final

In [52]:
train_img_unrow, train_label_final = reshape_input_image_label(train_images, 5216, train_labels)

In [54]:
test_img_unrow, test_label_final = reshape_input_image_label(test_images, 624, test_labels)

In [55]:
valid_img_unrow, valid_label_final = reshape_input_image_label(valid_images, 16, valid_labels)

In [56]:
np.shape(train_img_unrow)

(5216, 12288)

In [57]:
np.shape(test_img_unrow)

(624, 12288)

In [58]:
np.shape(valid_img_unrow)

(16, 12288)

In [59]:
np.shape(train_label_final)

(1, 5216)

In [60]:
np.shape(test_label_final)

(1, 624)

In [61]:
np.shape(valid_label_final)

(1, 16)

In [64]:
def standardize(img_unrow):
  img_final = img_unrow/255
  return img_final

In [65]:
train_img_final = standardize(train_img_unrow)

In [66]:
test_img_final= standardize(test_img_unrow)

In [67]:
valid_img_final = standardize(valid_img_unrow)

# **Model**

In [68]:
from keras import models
from keras import layers

In [72]:
baseline_model = models.Sequential()
baseline_model.add(layers.Dense(2, activation='relu', input_shape=(12288,)))
baseline_model.add(layers.Dense(1, activation='sigmoid'))
baseline_model.compile(optimizer='SGD',
                       loss='binary_crossentropy',
                       metrics=['recall'])
baseline_model_val = baseline_model.fit(train_img_final,
                                        train_label_final.T,
                                        epochs=6,
                                        batch_size=3,
                                        validation_data=(test_img_final, test_label_final.T))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/6
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 0.6096 - recall: 0.9958 - val_loss: 0.6906 - val_recall: 1.0000
Epoch 2/6
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.5708 - recall: 0.9994 - val_loss: 0.6964 - val_recall: 1.0000
Epoch 3/6
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.5712 - recall: 1.0000 - val_loss: 0.7029 - val_recall: 1.0000
Epoch 4/6
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.5661 - recall: 1.0000 - val_loss: 0.6936 - val_recall: 1.0000
Epoch 5/6
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 0.5761 - recall: 1.0000 - val_loss: 0.6962 - val_recall: 1.0000
Epoch 6/6
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.5743 - recall: 1.0000 - val_loss: 0.6990 - val_recall: 1.0000
